# this is case 3

we tried diarizing and stuff but it seems too overly complicated. the idea was to maybe make the speaker say something else instead of the profane word they used by an ai model trained on theri voice, if we could diarize it, we could identify what that speaker is, then figure out their dialoges from subtitles by matching with the diarizing output, then cut that part of audio out from the episode, and simlilarly make out a collection of data for that audio, and then train our model on it and instead of saying "fuck" you could have the model generate a different word like "freak". 

that could work but it seems far too much work. 

we used whisper for this. so in case we do not have subtitles, then yes, we could use whisper to generate a transcript and then remove profanity. it should work for the entire project anyway coz you can generate srt files from whisper and then thats the same thing

we also tried using whisper x to diarize but it didnt really work out and time spent to install and fix the whole thing would be too much effort for very less returns

In [9]:
# so now let us try instead to just remove the audio for the profane word, which should be simple enough to do, for many shows, including say severance

In [10]:
# from the subtitle well get start and end time of a dialogue. lets mute it using ffmpeg

In [2]:
import time

In [3]:
# say start time is 00:00:00, end time is 00:00:05
import pysrt

# subs = pysrt.open('ep1.srt')

In [4]:
subs = pysrt.open("ep1.srt")
mute_sections = ""
for sub in subs:
    start = sub.start.to_time()
    end = sub.end.to_time()
    mute_sections += f"volume=enable='between(t,{start.hour*3600+start.minute*60+start.second},{end.hour*3600+end.minute*60+end.second})':volume=0,"
# Remove last comma
mute_sections = mute_sections.rstrip(",")

In [1]:
start_time = subs[0].start.to_time()
end_time = subs[0].end.to_time()

NameError: name 'subs' is not defined

In [14]:
%pip install ffmpeg-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
# lets mute that audio in the video using ffmpeg
import ffmpeg

In [7]:
input_video = "ep1.mkv"
output_video = "ep1_muted.mkv"

In [ ]:
# entire video directly. its larger smh
ffmpeg.input(input_video).output(output_video, af=mute_sections).run()

In [4]:
ffmpeg.input("ep1.aac").output("ep1_muted.aac", af=mute_sections, acodec="aac").run()

(None, None)

In [4]:
video = ffmpeg.input(input_video)
audio = ffmpeg.input("ep1_muted.aac")

ffmpeg.output(
    video, audio, "ep1_audio_muted_remux.mkv", vcodec="copy", acodec="aac"
).run()

(None, None)

In [ ]:
import ffmpeg
import pysrt

# Input and output files
input_video = "ep1.mkv"
extracted_audio = "ep1_audio_method2.mka"  # Extracted audio format remains unchanged
muted_audio = "muted_audio_method2.mka"
output_video = "ep1_muted_method2.mkv"

# Step 1: Extract the original audio (without re-encoding)
ffmpeg.input(input_video).output(extracted_audio, vn=None, acodec="copy").run()

# Step 2: Parse subtitles and generate mute filter
subs = pysrt.open("ep1.srt")
mute_sections = [
    f"volume=enable='between(t,{sub.start.ordinal / 1000},{sub.end.ordinal / 1000})':volume=0"
    for sub in subs
]
mute_filter = ",".join(mute_sections)

# Step 3: Apply mute filter to extracted audio
ffmpeg.input(extracted_audio).output(muted_audio, af=mute_filter, acodec="copy").run()

# Step 4: Remux muted audio with original video (no re-encoding)
video = ffmpeg.input(input_video)
audio = ffmpeg.input(muted_audio)
ffmpeg.output(video, audio, output_video, vcodec="copy", acodec="copy").run()

print("Processing complete. Check", output_video)